# Piru Music Composer

In [1]:
import guitarpro
import pandas as pd
import numpy as np
import math
import os

## Loading data

Since GuitarPro format express the music in terms of fret on chords, we should first derive the notes from them


Now if we want to convert this "tablature" into proper music we can use the following way

In [ ]:
NOTES = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']

A4_MIDI_VALUE = 69

class Note:
    """
    Represents a musical note 
    
    Parameters
    ---------------
    midi_value: int (optional default None)
            If given instantiate the note from its midi value
    gpnote: guitarpro.models.Note (optional)
            If given, initialize the note to GuitarPro equivalent
            Must be given with tuning
    tuning: Array of guitarpro.models.GuitarString (optional)
            The tuning of the GuitarPro track from where the note comes from
    """
    def __init__(self, midi_value=None, gpnote = None, tuning=None):
        self.letter = 'A'
        self.octave = 4
        self.freq = 440
        self.midi_value = A4_MIDI_VALUE
        if midi_value:
            self._from_midi(midi_value)
            return
        
        if gpnote:
            if not tuning:
                raise ValueError("gpnote should always be given with the instrument tuning")
            self._from_gpnote(gpnote, tuning)

        
    def _from_gpnote(self, gpnote=None, tuning=None):
        midi_value = None
        for s in tuning:
            if gpnote.string == s.number:
                midi_value = s.value + gpnote.value
                break
        
        if midi_value == None:
            raise ValueError("Error in converting GPnote: string %i, fret %i. Please check tuning is correct" % (gpnote.string,gpnote.value))
        self.midi_value = midi_value
        self._from_midi(midi_value)
        
    def _from_midi(self, midi_value):        
        self.octave = math.floor((midi_value - A4_MIDI_VALUE)/len(NOTES))+4
        self.letter = NOTES[(midi_value - A4_MIDI_VALUE) % len(NOTES)]
        
        
    def __str__(self):
        return self.letter + str(self.octave)

def get_tuning(gp_tuning):
    tuning_str = ''
    for s in gp_tuning:
        tuning_str=str(Note(s.value))+tuning_str
    return tuning_str

We will learn a corpus using LSTM recurrent network.

In a first attempt we will only learn guitar and will only try to learn the succession of notes or chords disregarding the rythm.

A note is denomated by its MIDI value while a chords is an assemblage of MIDI values joined by dot

> Ex: 59.54.47 is a A power chord

In [19]:
DF_COLUMNS = ['Beat','Duration', 'Note' ]
DF_META = ['Song','Author','Genre']

def song_to_guitar_dataframe(file_path, append_meta=False):
    df = pd.DataFrame(columns = DF_COLUMNS+ (DF_META if append_meta else []))
    df = df.set_index(['Beat','Duration'])
    i = 0
    meta = file_path.split('/')
    
    title = meta[-1][:-4]
    author = meta[-2].replace('_',' ')
    genre = meta[-3]
    
    title = title.replace(author, '').strip(' -')
    
    
    print('*******', title,'-', author,'-', genre)
    has_guitar=False
    with open(file_path,'rb') as file:
        song = guitarpro.parse(file)
    for track in song.tracks:
        if not track.isPercussionTrack:
            tuning = get_tuning(track.strings)
            if 24 <= track.channel.instrument <= 30:
                current='Guitar'
        else:
            continue
        for measure in track.measures:
            for voice in measure.voices:
                p_beat = 0
                for beat in voice.beats:
                    note_str = ''
                    for note in beat.notes:
                        if note.type == guitarpro.NoteType.dead:
                            note_str='x.'
                            break
                        if note.type == guitarpro.NoteType.tie:
                            continue
                        
                        note_str += str(Note(gpnote = note, tuning = track.strings).midi_value)+'.'
                        
                    if note_str == '':
                        continue
                    df.loc[(beat.start,beat.duration.value),'Note'] = note_str[:-1]

                    i+=1
        
    return df.sort_values('Beat')  

To facilitate this first round of leaning we will stick to one artist and have elected nirvana for its relative simplicity.

We save those data into a csv file in order to load it faster next time we want to run a training

In [20]:
# Load a dataset

df = pd.DataFrame(columns=DF_COLUMNS)
directory = '/media/nico/Code/Sources/PiruCompose/data/Guitar Pro Tabs #1/Punk_Grunge/Nirvana/'
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".gp3") or filename.endswith(".gp4") or filename.endswith(".gp5"): 
        df = df.append(song_to_guitar_dataframe(directory + filename))

df = df.set_index(['Beat','Duration'])        
df.to_csv(directory[:-1]+'-guitars.csv')

df.describe()

******* Downer (2) - Nirvana - Punk_Grunge
******* Marigold - Nirvana - Punk_Grunge


/home/nico/.local/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


******* Heart-Shaped Box - Nirvana - Punk_Grunge
******* School (2) - Nirvana - Punk_Grunge
******* Radio Friendly Unit Shifter - Nirvana - Punk_Grunge
******* Lithium (Live Version) - Nirvana - Punk_Grunge
******* Moist Vagina - Nirvana - Punk_Grunge
******* Breed (5) - Nirvana - Punk_Grunge
******* Scentless Apprentice - Nirvana - Punk_Grunge
******* D7 - Nirvana - Punk_Grunge
******* Lithium - Nirvana - Punk_Grunge
******* Sappy (Acoustic) - Nirvana - Punk_Grunge
******* Something In The Way (MTV Unplugged) - Nirvana - Punk_Grunge
******* Dumb (2) - Nirvana - Punk_Grunge
******* Plateau (2) - Nirvana - Punk_Grunge
******* Frances Farmer Will Have Her Revenge On Seattle (2) - Nirvana - Punk_Grunge
******* Smells Like Teen Spirit (7) - Nirvana - Punk_Grunge
******* Floyd the Barber - Nirvana - Punk_Grunge
******* The Man Who Sold The World - Nirvana - Punk_Grunge
******* On A Plain (2) - Nirvana - Punk_Grunge
******* Jesus Doesn't Want Me For A Sunbeam (Live Rock Version) - Nirvana - 

## Shaping data and building network

We split the data in sequence of 100 successive notes or chords, the goal of the training is to learn the next note or chord

The data is then turn into binary using One Hot Encoder.
It means each sample actually have as many column as the number or distinct note or chords found in the corpus and its row is felt with 0 except for the note or chord it represent, which have a 1

In [22]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer

sequence_length = 100

network_input = []
network_output = []

onehotenc_notes = pd.get_dummies(df.iloc[:,0])
n_features = len(onehotenc_notes.columns)
print('Sequence length', sequence_length, 'nb features',n_features)
# create input sequences and the corresponding outputs
for i in range(0, len(df) - sequence_length, 1):
    sequence_in = onehotenc_notes.iloc[i:i + sequence_length,:].values
    sequence_out = onehotenc_notes.iloc[(i + sequence_length):(i + sequence_length+1),:].values
    network_input.append(sequence_in)
    network_output.append(sequence_out)
n_patterns = len(network_input)
network_input = np.array(network_input)
network_output = np.array(network_output)

print(np.array(network_input).shape, np.array(network_output).shape)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, n_features))
network_output = np.reshape(network_output, (n_patterns, n_features))

print(network_input.shape, network_output.shape)


Sequence length 100 nb features 268
(37088, 100, 268) (37088, 1, 268)
(37088, 100, 268) (37088, 268)


We build a 2 layers LSTM network to learn how to play Nirvana.
Among the non standard parameters one should note:

- We use `return_sequences=True` since we eventually want to produce sequences of note, not just the next one
- We use the `categorical_crossentropy` loss function as our task is actually to predict the discrete category of representing the followings notes or chords
- `softmax` activation can be questionned, we use it to sharpen the choice of one specific note or chords but it would worth exploring a more classical `sigmoid` activation
- `adam` optimizer is nowaday the most classical for achieving faster learning

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_features))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 256)          537600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)              

We save the model after each epoch so that we can:

- Stop and start it again (or restart it from this stage in case of computer crash)
- Evaluating the model as it is training (See the second notebook **Piru Music Composer-Compose only**)

In [ ]:
filepath = "output/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

Epoch 1/200
37088/37088 [==============================] - 1164s 31ms/step - loss: 2.7126
Epoch 2/200
37088/37088 [==============================] - 1246s 34ms/step - loss: 1.7162
Epoch 3/200
37088/37088 [==============================] - 1205s 32ms/step - loss: 1.2840
Epoch 4/200
37088/37088 [==============================] - 1067s 29ms/step - loss: 1.0284
Epoch 5/200
37088/37088 [==============================] - 1058s 29ms/step - loss: 0.8392
Epoch 6/200
37088/37088 [==============================] - 1108s 30ms/step - loss: 0.6902
Epoch 7/200
37088/37088 [==============================] - 1293s 35ms/step - loss: 0.5837
Epoch 8/200
37088/37088 [==============================] - 1260s 34ms/step - loss: 0.5145
Epoch 9/200
37088/37088 [==============================] - 1175s 32ms/step - loss: 0.4400
Epoch 10/200
 7488/37088 [=====>........................] - ETA: 15:22 - loss: 0.4045